# Bloqade High-Performance Capabilities

John Long, *Scientific Software Developer*

This notebook attempts to display some of the High-Performance capabilities that Bloqade has built-in. While prior examples may be designed to run in their own notebooks the nature of some of the capabilities presented (requiring the restarting or potential modification of kernels via their startup arguments) means this particular notebook is **NOT DESIGNED TO BE RUN IN ITS ENTIRETY**. If you would like to follow along with certain examples you will need to duplicate them outside the notebook in a separate Julia session. 

First though, it's worth taking a look at how Bloqade simulates things from a birds-eye view to understand the benefits of each High-Performance method and why the exist in the first place:

1. You define a Hamiltonian expression of interest (usually with `rydberg_h`)
2. The Hamiltonian expression, along with a register (you can think of this as the statevector), and amount of time to simulate evolution for, are passed into a `SchrodingerProblem`
3. The `SchrodingerProblem` "lowers" or reduces the expression into a series of different Sparse Matrix formats that represent certain terms in the Hamiltonian
4. The register that was passed in earlier is repeatedly multiplied against these matrices via Sparse-Matrix Dense-Vector (SpMV) multiplication as part of the Runge-Kutta solver.

Can we optimize any part of these steps? Yes! The optimizations essentially fall into two categories:

* Memory Efficiency - Store a smaller amount of data while sacrificing as little accuracy as possible
* Execution Efficiency - Keeping the format of the problem the same but speeding up the actual execution

## Memory Efficiency - Blockade Subspace

We can simulate larger system sizes by realizing that there are scenarios where it isn't necessary to preserve all $2^n$ possible states. If two atoms blockade each other by being close enough and attempting to simultaneously be in the Rydberg state ($\langle|rr\rangle$), the probability of getting a double Rydberg Excitation state is impossible. Thus we can safely get rid of these states but we should still exercise care because of the possibility of dropping non-trivial contributions to the dynamics.

The actual code and a more detailed explanation can be found in this accompanying notebook: [Blockade Subspace](Section_V_Blockade_Subspace.ipynb)

## Execution Efficiency - Multithreading and GPU Support

Faster simulation times in Bloqade are achievable through multi-core CPUs via multithreading support and on Nvidia GPUs via CUDA support.

This is all thanks to Julia's support for *multiple dispatch*, which allows you to define the same function but for different types. If the register is changed to a GPU-specific type, then all arithmetic operations involving the register call their GPU variants. 

In a similar vein, for multithreading the sparse matrices will now invoke a parallelized matrix-vector multiplication function instead of the default single-threaded version simply through some automatic type wrapping that occurs behind the scenes.

### Multithreading

Before showing the speedup attainable via multi-threading let's see a simulation run without it. In the cell below has code to measure the run time for the simulation of an adiabatic sweep on a 4 x 4 lattice. This leads to two possible final states in equal superposition, both with an alternating Rydberg-Ground pattern (although the orders are flipped):

*NOTE:* Owing to Julia's method of precompilation, the first time you run any new function there will be some overhead. The function should be run a second time to give the most accurate benchmarking results. If you are looking avoid this precompilation when running a computationally intensive simulation on the first run, you can run a significantly smaller problem (say, one atom) with the same function calls and this should get the precompilation out of the way.

In [ ]:
using Bloqade

# can show speedup with different backends
function lattice_sweep_benchmark()

    # 4x4 lattice of atoms with adiabatic sweep to create something *like* Z2 ordered phase,
    # although degeneracy present due to even number of sites
    atom_distance = 6.7;
    atoms = generate_sites(SquareLattice(), 4, 4, scale = atom_distance);

    # define waveforms
    Ω_max = 2π * 2.5;
    Δ_val = 2π * 10.0;
    total_time = 3.0;
    time_ramp  = total_time * 0.083;
    clocks = [0, time_ramp, total_time - time_ramp, total_time];

    Ω = piecewise_linear(clocks = clocks, values = [0.0, Ω_max, Ω_max, 0.0]); 
    Δ = piecewise_linear(clocks = clocks, values = [-Δ_val, -Δ_val, Δ_val, Δ_val]);
    ϕ = constant(;duration = total_time, value = π);

    h = rydberg_h(atoms; Ω=Ω, Δ=Δ, ϕ=ϕ);

    reg = zero_state(16);

    total_time = 3.0
    println("Creating SchrodingerProblem...")
    prob = SchrodingerProblem(reg, total_time, h;algo=DP8());
    println("Running Emulation...")
    # print the amount of time it took to run this
    # Due to Julia's precompilation, the first time this runs will incur a precompilation overhead. 
    # Running a second time will give a more accurate result.

    # Benchmarking with @time is generally discouraged in Julia but for our purposes is useful enough to show 
    # the speedup of multithreading
    @time emulate!(prob);

    return nothing

end

Now let's enable multithreading. Multithreading is supported via the `BloqadeExpr` subpackage which supports two different *backends*, each designed for slightly different simulation scenarios.

* `ThreadedSparseCSR` - Minimal/no overhead (other than thread spawning) and is good for balanced matrices (number of non-zeros per row remains consistent)
* `ParallelMergeCSR` - Pay an up front cost but ensure optimal balancing of matrix-vector multiplication workload across multiple cores (avoid problem of having one core essentially stop other cores from doing useful work), good for unbalanced matrices (possible when employing the blockade subspace optimization OR you end up defining a phase component.)

One way to enable multithreading is the following:

1. Launch Julia with the desired number of threads to use through `julia -t num_of_threads`
2. Create a Julia environment (this can be done by creating an empty folder for your project and upon navigating to it, going into the package management mode of a Julia session and typing `activate .` . The creation of the new environment should be successful if you see the text before `pkg>` turn into the name of the folder itself (e.g. `(my-empty-folder)>`))
3. Add `Bloqade` and `BloqadeExpr` to the environment (while you still see `pkg>` just type `add Bloqade` and `add BloqadeExpr` afterwards)
4. Add a file called `LocalPreferences.toml` to the same folder you were working in (your directory structure should look something like the following below):

```
.
├── Mainfest.toml
├── Project.toml
└── LocalPreferences.toml

```

`LocalPreferences.toml` should have the following as its contents:

```
[BloqadeExpr]
backend = "ParallelMergeCSR"
```
Where backend can be set to `"ParallelMergeCSR"`, `"ThreadedSparseCSR"` or `"BloqadeExpr"` (`BloqadeExpr` is the default, single-threaded backend. Even if you launch Julia with multiple threads, Bloqade's default behavior is single-threaded and you must explicitly specify that you want multithreading by supplying a different backend).

5. Ensure that Bloqade has the right multithreading backend via `julia> using BloqadeExpr` and then `julia> BloqadeExpr.backend`. You should see the same string you set in the `LocalPreferences.toml` printed out. 


Another way that doesn't require you create any additional files (but does require restarting your Julia session is the following):

1. Add `Bloqade` and `BloqadeExpr` (see the instructions above, step 2. Note that while this procedure doesn't require a custom Julia environment, it is considered good practice to do so.)
2. Exit the package management mode and type in `julia> using BloqadeExpr`
3. Set your desired backend via `julia> BloqadeExpr.set_backend("backend_string_here")` where "backend_string_here" can be any of the options listed in step 4 above.
4. You will see a notice requesting you restart Julia. Exit the session.
5. Upon restarting the session, launch Julia via `julia -t num_of_threads`
    * As an additional sanity check, you can ensure Julia sees it has multiple threads available to it by importing `Base.Threads` and invoking the `nthreads()` function
6. Type `julia> using BloqadeExpr` and then `BloqadeExpr.backend`.

If you see the backend you provided earlier printed, Bloqade will now have multithreading enabled.


As an exercise, re-run the `lattice_sweep_benchmark()` with multithreading enabled and see how much the performance improves. You should be able to achieve a ~3x speedup over the single threaded version!

You can find some more information about multithreading in Bloqade via the documentation here: https://queracomputing.github.io/Bloqade.jl/dev/multithreading/

### GPU Support

All that needs to be done to run any simulations on a GPU is to move the `SchrodingerProblem` onto the GPU first, run the simulation, and then offload it! Even easier than multithreading, although there are some things that should be kept in mind:

* The size of your problem is now limited to how much memory your GPU has
* There is a one-time cost in terms of loading and off-loading your problem from GPU memory back into main memory
  * But, even with this cost a great deal of performance benefit can be extracted!
* Currently, Bloqade only supports single GPUs but multi-node/multi-GPU support is planned in the future

To see the performance boost in action as well the procedure for using the GPU, we'll start by defining a problem for benchmarking (in this case, a 10-atom 1D chain with an adiabatic sweep applied to it).

Once again, because of Julia's precompilation methods it's best to run this function twice so the precompilation overhead is not factored in as part of the execution time.

In [ ]:
using Bloqade

function chain_run_time_CPU()

    # define an adiabatic sweep to create a Z2 ordered phase on a chain of ten atoms

    total_time = 3.0;
    Ω_max = 2π * 4;
    Ω = piecewise_linear(clocks = [0.0, 0.1, 2.1, 2.2, total_time], values = [0.0, Ω_max, Ω_max, 0, 0]);

    U1 = -2π * 10;
    U2 = 2π * 10;
    Δ = piecewise_linear(clocks = [0.0, 0.6, 2.1, total_time], values = [U1, U1, U2, U2]);

    nsites = 14
    atoms = generate_sites(ChainLattice(), nsites, scale = 5.72)

    h = rydberg_h(atoms; Δ, Ω)
    reg = zero_state(nsites);
    println("Creating SchrodingerProblem")
    problem = SchrodingerProblem(reg, total_time, h);

    println("Measuring CPU time")
    @time emulate!(problem)
    
    # need to bring the problem back into main memory
    return nothing
end

To run this on the GPU, you can take a look at the example below:

In [ ]:
using Bloqade
using Adapt
using CUDA

function chain_run_time_GPU()

    # define an adiabatic sweep to create a Z2 ordered phase on a chain of fourteen atoms

    total_time = 3.0;
    Ω_max = 2π * 4;
    Ω = piecewise_linear(clocks = [0.0, 0.1, 2.1, 2.2, total_time], values = [0.0, Ω_max, Ω_max, 0, 0]);

    U1 = -2π * 10;
    U2 = 2π * 10;
    Δ = piecewise_linear(clocks = [0.0, 0.6, 2.1, total_time], values = [U1, U1, U2, U2]);

    nsites = 14
    atoms = generate_sites(ChainLattice(), nsites, scale = 5.72)

    h = rydberg_h(atoms; Δ, Ω)
    reg = zero_state(nsites);
    println("Creating SchrodingerProblem")
    problem = SchrodingerProblem(reg, total_time, h; algo=DP8());

    # Take advantage of multiple dispatch here, objects that are CuArrays 
    # automatically have GPU routines applied to them versus CPU implementation
    println("Measuring GPU load/execution/unload time")
    @time begin
        gpu_problem = adapt(CuArray, problem)
        emulate!(gpu_problem)
        offloaded_problem = adapt(Array, gpu_problem)
    end
    
    # need to bring the problem back into main memory
    return nothing

end

Notice these lines:
```julia
gpu_problem = adapt(CuArray, problem)
emulate!(gpu_problem)
offloaded_problem = adapt(Array, gpu_problem)
```

We literally just take the `SchrodingerProblem`, `adapt` it onto the GPU, let the simulation run and then take it off for further analysis!

In the interest of a comparable benchmark, note that the `@time` macro includes the loading and unloading times.

You should find that there is still a healthy performance increase.

## Conclusion

You've now been exposed to Bloqade's High Performance capabilities.

You've seen:
* A birds-eye view of how Bloqade performs simulations
* Choices for memory and execution efficiency
* The different multithreading backends and their pros/cons
* How to take advantage of Nvidia GPUs

Now you can simulate many-body neutral atom dynamics *at scale* (: 